# **LENDING CLUB CASE STUDY** 
## **Group Members :**
### **1. Karthick Chetti** - Group Fecilitator
### **2. Anirudh KVC** - Collaborator

## **Index of Notebook**
1. Objectives
2. Understanding Data
3. Data Cleaning
4. Selecting columns based on business objectives
5. Missing Value Imputation
6. Understanding variables and their datatypes


## **1. Objectives :**
1. Understanding the driving factors behind loan default thereby aiming to reduce credit loss 
2. Data Driven inferences to support the driving factors

In [ ]:
# Load the libraries
import pandas as pd #To work with dataset
pd.set_option("display.max_columns", 100)
import numpy as np #Math library
import seaborn as sns #Graph library that use matplot in background
import matplotlib.pyplot as plt #to plot some parameters in seaborn
from matplotlib.ticker import PercentFormatter # for percentage

In [ ]:
# Importing data from loan.csv
loan_df=pd.read_csv("loan.csv")

### **2. Understanding Data** 

In [ ]:
loan_df.info(verbose=True,show_counts=True)
init_cols=len(loan_df.columns)

#### We can see that there are some null values in the columns and there are too many columns to analyze, hence measures have to be taken to clean data based on *business objectives* and based on *statistics*

### **3. Data Cleaning** : Based on Business Objective
### Filtering out data based on objective, since current loans are not of interest for the business objective

In [ ]:
# As we are interested in understanding charged off customers loan status as current is not useful for the analysis
# Filtering data accorgingly
loan_df = loan_df[~ (loan_df["loan_status"] == "Current")]

### **3. Data Cleaning** : A) Removing Columns with high percentage of null values

In [ ]:
#Removing columns having 50% null values in them as such columns are not useful for analysis
lc_df=loan_df.dropna(axis=1,thresh=int(0.5*len(loan_df)))

In [ ]:
#Printing number of columns removed
removed_columns=len(loan_df.columns)-len(lc_df.columns)
print(f"{removed_columns} columns were removed from original dataframe due to high amount of missing values in them")

### **3. Data Cleaning** : B) Removing Columns with same values which are redundant for analysis

In [ ]:
#### It is observed that some columns have only one value(mostly 0) throughout the rows, which are not useful for the analysis
dropped_col=[]
for col in lc_df.columns:
    if len(lc_df[col].unique())==1:
        dropped_col.append(col)
print(f"{len(dropped_col)} columns are dropped in the dataframe due to same values repeating throughout the column")
lc_df=lc_df.drop(dropped_col,axis=1)

### **3. Data Cleaning** : C) Removing Columns with either 0s or missing values in them

In [ ]:
for c in lc_df.columns:
    if len(lc_df[c].unique())==2:
        print(lc_df[c].unique())

In [ ]:
no_uniq_cols=["collections_12_mths_ex_med","chargeoff_within_12_mths","tax_liens"]
lc_df.drop(no_uniq_cols,axis=1,inplace=True)
print(f"{len(no_uniq_cols)} columns are dropped since they don't have meaningful unique values for analysis")

In [ ]:
exis_cols=len(lc_df.columns)
print(f"As a result of data cleaning, out of {init_cols} columns, {exis_cols} columns exist.\nHowever it is important to select the most relevant columns out of these {exis_cols} for analysis")

### **4. Selecting most relevant features as per business objective** 
### Additional features/columns will be added to the analysis in the future if it's found relevant

In [ ]:
# Key features
# id,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,verification_status,purpose,zip_code,add_state
# Here id is conidered for aggregation purpose
req_features=["id","loan_amnt","term","int_rate","installment","grade","sub_grade","home_ownership","emp_length","loan_status","annual_inc","verification_status","purpose","zip_code","addr_state","dti","open_acc","total_acc","delinq_2yrs","earliest_cr_line"]
lc_df = lc_df[req_features]

In [ ]:
# Having a look at the data
lc_df.head()

In [ ]:
# Understanding the data and data types
lc_df.info(verbose=True,show_counts=True)
lc_df.describe()

### **5. Missing value imputation**

### There are some columns which have missing values,Therefore understanding the number of missing values in these columns

In [ ]:
empty_cols=lc_df.columns[lc_df.isna().any()].tolist()
empty_vals=[lc_df[col].isna().sum() for col in empty_cols]
empty_per_dict=dict(zip(empty_cols,empty_vals))
print("The missing values in columns is :\n",empty_per_dict)

#### It is observed that in selected columns for analysis, 'emp_length' column has some missing data

In [ ]:
# As the number of missing data is less, missing value imputation can be done
# Checking the values in emp_length column
lc_df["emp_length"].value_counts()

In [ ]:
# As the most recurring value is 10+years, adding 10+ years whereever the data is missing
lc_df["emp_length"] = lc_df["emp_length"].fillna("10+ years")

### **6. Understanding variables and their datatypes**

In [ ]:
#Understanding the datatypes in the dataframe
lc_df.info(verbose=True,show_counts=False)

In [ ]:
# Removing % and converting to float
lc_df["int_rate"]=lc_df["int_rate"].str.replace("%","").astype(float)

### **Understanding type of variable for analysis**
1. **Numeric Data** : <br>
loan_amt,int_rate,installment,annual_inc,dti

2. **Ordered Categorical Data** : <br>
term,grade,sub_grade,emp_length,loan_status,verification_status,open_acc,total_acc,delinq_2yrs,earliest_cr_line

3. **Unordered Categorical Data** : <br>
home_ownership,purpose,zip_code,addr_state

*id column can be used as a unique row identifier*

In [ ]:
#Declaring type of variables
numeric_data=["loan_amnt","int_rate","installment","annual_inc","dti"]
ordered_cat_data=["term","grade","sub_grade","emp_length","loan_status","verification_status","open_acc","total_acc","delinq_2yrs","earliest_cr_line"]
unordered_cat_data=["home_ownership","purpose","zip_code","addr_state"]

### **7. Data Distribution and Outlier Correction for Numeric Data**

In [ ]:
#Checking the distributions for numeric data
#Function to plot box plots taking dataframe and columns as input
def box_plot_col(df,cols):
    for col in cols:
        plt.boxplot(df[col])
        plt.xlabel(col)
        plt.show()
box_plot_col(lc_df,numeric_data)

### Outliers are detected in the box plot for some variables, it is important to remove outliers in the data based on a  IQR 

In [ ]:
# #Remove outliers based on threshold of 99%
# def remove_outliers(df,thresh,cols):
#     for col in cols:
#         ser=df[col]
#         P = np.percentile(ser, [100-thresh, thresh])
#         df = df[(ser > P[0]) & (ser < P[1])]
#     return df
        
# lc_df=remove_outliers(lc_df,99,numeric_data)
# removed_data_points=len(loan_df)-len(lc_df)
# print(f"{removed_data_points} data points were removed because they were outliers in the numeric data columns")

In [ ]:
#Remove outliers based on IQR
def remove_outliers(df,cols):
    for col in cols:
        ser=df[col]
        Q1 = ser.quantile(0.25)
        Q3 = ser.quantile(0.75)
        IQR = Q3 - Q1
        df = df[~(ser > (Q3 + 1.5 * IQR))]
    return df
lc_df=remove_outliers(lc_df,numeric_data)
removed_data_points=len(loan_df)-len(lc_df)
print(f"{removed_data_points} data points were removed because they were outliers in the numeric data columns")

### **8. Data Distribution for Ordered Categorical Variables**

### Since deriving insights on entire data is not useful and does not give relevant insights with respect to our business objective 
### We are visualizing the insights with respect to **fully paid** and **charged off** customers

In [ ]:
# Functions for plotting bar chart for categorical variables

def plot_bar_cat(df,index_name,column_name,width,x_lbl_rotation):
    pivot=pd.pivot_table(lc_df,values="id",index=index_name,columns=column_name,aggfunc=lambda x: len(x.unique()))
    plt.figure(figsize=(15,5))
    x_label = pivot.index
    x_data = np.arange(len(x_label)) # create values from 0 to len(x_label) - 1
    #height_data = (df[df["loan_status"]=="Fully Paid"][column_name].value_counts().values,df[df["loan_status"]=="Charged Off"][column_name].value_counts().values)
    chart=[]
    for j,i in enumerate(pivot.columns):
        chart.append(plt.bar(x_data + j*(width), pivot[i],width=width,align="edge"))
    plt.xlabel(index_name,fontsize=22)
    plt.ylabel("Frequency",fontsize=22)
    x_label_pos = x_data + (len(pivot.columns)*width)/2
    plt.xticks(x_label_pos ,x_label,fontsize=15,rotation = x_lbl_rotation)
    plt.yticks(fontsize=15)
    plt.legend(chart,pivot.columns,fontsize=15)
    plt.show()

ord_cat_plot_cols=ordered_cat_data.copy()
ord_cat_plot_cols.remove("earliest_cr_line")
ord_cat_plot_cols.remove("loan_status")
for col in ord_cat_plot_cols:
    if col not in ["earliest_cr_line","loan_status","open_acc","total_acc"]:
        plot_bar_cat(lc_df,col,"loan_status",0.1,45)

### **Inferences from univariate analysis on ordered categorical variables**
1. The number of charged off people are more when the term of loan is 36 months as compared to 60 months *(Probably paying more installment amounts might be difficult)*<br>
2. Very few people are interested to take loans beyond D grade. In B,C & D loan grades,the charged off customers remain almost the same but the number of fully paid customers reduce *(Probably due to increase in risk and volatality of these loan grades, Source - https://www.lendingclub.com/foliofn/rateDetail.action)*<br>
3. Majority of the charged of customers either have 10+ years of experience *(Probably due to other existing credit lines) or they have less than or equal to 1 year of experience*(Probably due poor financial management)*<br>
4. Majority of customers have no history of deliquency within 2 years, *(Probably because LC might be cautious in providing loans to people having history of deliquency)*

In [ ]:
#Unordered categorical variables
for i in unordered_cat_data:
    x_data = np.arange(1,len(lc_df[i].value_counts())+1)
    y_data = lc_df[i].value_counts().values
    plt.loglog(x_data,y_data)
    plt.xlabel("log (Rank)")
    plt.ylabel("log (Frequency)")
    plt.title(i)
    plt.show()

In [ ]:
# From the rank vs frequency plots it can be inferred that purpose follows nearly power law distribution

In [ ]:
# Plotting bar chart for unordered categorical variables
for col in unordered_cat_data:
    if col not in ("zip_code","addr_state"):
        plot_bar_cat(lc_df,col,"loan_status",0.1,45)

In [ ]:
# Inferences from unordered categorical univariate analysis
# Majority of the charged off customers are on rent or mortgage
# Most of the charged off customers have taken loans for debt consolidation

### **9. Derived Metrics and Binning for Ordered Categorical Variables**

In [ ]:
# Functions for plotting graphs wrt columns (numerical) as percentage
def plot_hist(df,column_name,bins):
    plot_data = (df[df["loan_status"]=="Fully Paid"][column_name],df[df["loan_status"]=="Charged Off"][column_name])
    #weight_data= (np.ones(len(plot_data[0])) / len(plot_data[0]+plot_data[1]),np.ones(len(plot_data[1])) / len(plot_data[0]+plot_data[1]))
    #weight_data= (np.ones(len(plot_data[0])) / len(plot_data[0]),np.ones(len(plot_data[1])) / len(plot_data[1]))
    plt.figure(figsize=(15,5))
    plt.hist(plot_data,bins=bins)
    plt.xlabel(column_name,fontsize=22)
    plt.ylabel("Frequency",fontsize=22)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    #plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
    plt.legend(("Fully Paid","Charged Off"),fontsize=15)
    plt.show()
    

In [ ]:
# Change bin_dict variable for binning
bin_dict = {"loan_amnt":np.linspace(0,30000,num=9),"int_rate":np.linspace(0,25,num=6),"annual_inc":np.linspace(4000,200000,num=11),"dti":np.linspace(0,30,num=7),"installment":np.linspace(0,800,num=9)}
plot_hist(lc_df,"loan_amnt",bin_dict["loan_amnt"])
plot_hist(lc_df,"int_rate",bin_dict["int_rate"])
plot_hist(lc_df,"annual_inc",bin_dict["annual_inc"])
plot_hist(lc_df,"dti",bin_dict["dti"])
plot_hist(lc_df,"installment",bin_dict["installment"])


In [ ]:
# As the loan amount is increasing the number of fully paid customers are decreasing
# Most of the charged off customers are from 10 to 15 % interest rate
# As the annual income of the customers is increasing the number of fully paid customers are also increasing probably because of high income to reapy the loan
# As the installment amount is increasing the number of fully paid customers are reducing

In [ ]:
#Derived Metric 1 : Employee length bins
#Employee length can be binned based on distribution into lesser categories (<1 yr,1-3yr,4-6yr,7-9yr,>10yr)
def bin_emp_len(x):
    if x in ["< 1 year"]:
        return "<1 year"
    elif x in ["1 year","2 years","3 years"]:
        return "1-3 years"
    elif x in ["4 years","5 years","6 years"]:
        return "4-6 years"
    elif x in ["7 years","8 years","9 years"]:
        return "7-9 years"
    elif x in ["10+ years"]:
        return ">10 years"
lc_df['emp_length']=lc_df['emp_length'].apply(bin_emp_len)

In [ ]:
# Derived Metric 2 : Total Closed Credit lines
lc_df['closed_acc']=lc_df['total_acc']-lc_df['open_acc']
# Removing the data point as the number of closed accounts in negative
lc_df = lc_df[~lc_df["closed_acc"]<0]
lc_df["closed_acc"].describe()
def label_generator(x):
    lab = []
    for i in range(0,len(x)-1):
        lab.append(str(x[i]) + " to " + str(x[i+1]))
    return lab
# Binning the closed accounts
lc_df["bin_closed_acc"] = pd.cut(lc_df["closed_acc"],bins=np.arange(0,90,15),labels=label_generator(np.arange(0,90,15)),include_lowest=True)
lc_df["bin_closed_acc"].value_counts()
# plot_bar_cat(lc_df,"bin_closed_acc","loan_status",0.2,0)

In [ ]:
# Derived Metric 3 : Extract year from earliest_cr_line
#lc_df['last_credit_line_year'] = pd.to_datetime(lc_df["earliest_cr_line"],format="%b-%y").dt.year
def year(x):
    y_int=int(x.split("-")[1])
    if y_int>=0 and y_int<=11:
        y_int = y_int + 2000
    else:
        y_int = y_int + 1900
    return y_int
lc_df['ear_cre_li_year'] = lc_df["earliest_cr_line"].apply(year)
lc_df["ear_cre_li_year"].describe()

In [ ]:
#Derived Metric 4 : Compare Installment to Monthly Income
lc_df['ins_div_mnth_inc'] = (lc_df['installment']*100)/(lc_df['annual_inc']*12)

In [ ]:
# df['bins'] = pd.cut(x=df['number'], bins=[1, 20, 40, 60, 80, 100],
#                     labels=['1 to 20', '21 to 40', '41 to 60',])

In [ ]:
# Binning numerical features
binned_num = []
for feat,b in bin_dict.items():
    col = "bin_" + feat 
    binned_num.append(col)
    lc_df[col] = pd.cut(lc_df[feat],bins=b,labels=label_generator(b),include_lowest=True)
plot_bar_cat(lc_df[lc_df["loan_status"]=="Charged Off"],"purpose","bin_int_rate",0.1,45)
plot_bar_cat(lc_df[lc_df["loan_status"]=="Charged Off"],"emp_length","open_acc",0.1,45)

In [ ]:
#Univariate Analysis

In [ ]:
# Univariate Analysis
# Functions for plotting graphs wrt columns (numerical) as percentage
def plot_hist(df,column_name,bins):
    plot_data = (df[df["loan_status"]=="Fully Paid"][column_name],df[df["loan_status"]=="Charged Off"][column_name])
    #weight_data= (np.ones(len(plot_data[0])) / len(plot_data[0]+plot_data[1]),np.ones(len(plot_data[1])) / len(plot_data[0]+plot_data[1]))
    weight_data= (np.ones(len(plot_data[0])) / len(plot_data[0]),np.ones(len(plot_data[1])) / len(plot_data[1]))
    plt.figure(figsize=(20,10))
    n,bin_data,patches=plt.hist(plot_data,color=("g","r"),weights=weight_data,bins=bins)
    plt.xlabel(column_name,fontsize=22)
    plt.ylabel("Percentage",fontsize=22)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
    plt.legend(("Fully Paid","Charged Off"),fontsize=15)
    plt.show()
    return n,bin_data,patches

In [ ]:
a1,b1,c1=plot_hist(lc_df,"loan_amnt",np.linspace(0,40000,num=9))
a2,b2,c2 = plot_hist(lc_df,"int_rate",np.linspace(0,25,num=6))
a3,b3,c3 = plot_hist(lc_df,"annual_inc",np.linspace(4000,6000000,num=100))

In [ ]:
# Insights from the histogram
# No loan is offered from 0 to 5 % interest rate

# wrt interest rate
#1) Around 48% of the charged of customers fall under interest rate of 10 to 15 percentage
#2) 85 % of the charged off customers have taken the loan at an interest rate of > 10 %

# wrt loan_amnt
#1) 82 % of the charged customers have taken the loan amount > 5000
#2) 50 % of fully paid customers have taken the loan amont < 10000

In [ ]:
plot_bar_cat(lc_df,"term","loan_status",0.2,0)
plot_bar_cat(lc_df,"home_ownership","loan_status",0.2,0)
plot_bar_cat(lc_df,"emp_length","loan_status",0.2,0)
plot_bar_cat(lc_df,"verification_status","loan_status",0.2,0)
plot_bar_cat(lc_df,"purpose","loan_status",0.2,45)
plot_bar_cat(lc_df,"addr_state","loan_status",0.2)

In [ ]:
# Insights from bar chart


In [ ]:
num_cols=[]
for col in lc_df.columns:
    if lc_df[col].dtype in ["int64","float64"]:
        num_cols.append(col)
num_lc_df_corr=lc_df[num_cols].corr()
num_lc_df = lc_df[num_cols]

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(num_lc_df_corr,cmap="viridis",annot=True)
plt.show()